# Entrenamiento de modelos con descenso del gradiente
Este notebook fue construido con recursos de [GitHub](https://github.com/ageron/handson-ml2).

*Entrenar* un modelo se refiere a configurar sus parámetros de forma que se ajusten de la mejor manera a los datos del conjunto de entrenamiento. Para esto se hace necesario definir una medida que nos indique qué tan bien (o mal) se está realizando ese ajuste de los parámetros, esta medida se suele llamar **función de costo**. Para tareas de regresión, la función de costo más utilizada es el error cuadrático medio o MSE (*Mean Squared Error*), definido como:

$$MSE(X,\theta)=\frac{1}{m}\sum_{i=1}^m(\theta^Tx^{(i)}-y^{(i)})^2$$

Donde $X$ representa nuestro conjunto de entrenamiento con $m$ muestras, $x^{(i)}$ representa los valores de las características de entrada para una muestra y $y^{(i)}$ la etiqueta o valor objetivo; y $\theta$ representa los parámetros del modelo. $\theta^Tx^{(i)}$ corresponde a las predicciones del modelo, por lo que también lo podríamos expresar como $\hat y$.

In [ ]:
import numpy as np
np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error # Función de sklearn para calcular el error cuadrático medio
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
# Generación de los datos lineales con algo de ruido
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0, 2, 0, 15])
plt.show()

Utilizando la ecuación normal para encontrar los mejores parámetros

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]  # Agregar x0 = 1 a cada muestra para permitir la multiplicación
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print(theta_best)

In [ ]:
predictions = X_b.dot(theta_best) # Realizar predicciones
mean_squared_error(predictions, y)

La solución del modelo por regresión lineal a través de la ecuación normal, se hace cada vez más compleja a medida que nos enfrentamos a conjuntos de datos más grandes o con más características, primero por una posible limitación de memoria RAM, o dado que podamos cargarlo, por tiempos elevados de procesamiento.

Es en estos casos en los que se deben usar diferentes estrategias de optimización, para llegar de manera eficiente a los parámetros ópitmos de nuestro modelo. Una de las estrategias más utilizadas es el descenso del gradiente o *gradient descent*.

## Descenso del gradiente

La idea principal es modificar los parámetros iterativamente hasta minimizar la función de costo, pero las modificaciones no son aleatorias, se hacen en la dirección descendente del gradiente local de la función de costo con respecto al vector de parámetros $\theta$, hasta llegar a un gradiente cero, indicando que hemos llegado a un mínimo de la función de costo.

El procedimeinto entonces es inicializar el vector $\theta$ con valores aleatorios (llamado *random intialization*), luego vamos mejorando poco a poco, buscando reducir la función de costo hasta que el algoritmo converge a un mínimo.

<img src="https://github.com/MoraRubio/ai-learning-resources/blob/main/supervised-learning-ml/src/gradient-descent.png?raw=true" width="230" align="middle"/>

El tamaño de cada salto, o de cada paso, está determinado por un hiperparámetro conocido como la tasa de aprendizaje o *learning rate*.

In [ ]:
eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

print(theta)

In [ ]:
predictions = X_b.dot(theta) # Realizar predicciones
mean_squared_error(predictions, y)

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
theta_path_bgd = []

def plot_gradient_descent(theta, eta, theta_path=None):
    m = len(X_b)
    plt.plot(X, y, "b.")
    n_iterations = 1000
    for iteration in range(n_iterations):
        if iteration < 10:
            y_predict = X_new_b.dot(theta)
            style = "b-" if iteration > 0 else "r--"
            plt.plot(X_new, y_predict, style)
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if theta_path is not None:
            theta_path.append(theta)
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 2, 0, 15])
    plt.title(r"$\eta = {}$".format(eta), fontsize=16)

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

plt.figure(figsize=(10,4))
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(132); plot_gradient_descent(theta, eta=0.1, theta_path=theta_path_bgd)
plt.subplot(133); plot_gradient_descent(theta, eta=0.5)

plt.show()

En el ejemplo anterior utilizamos todos los datos de nuestro conjunto de entrenamiento en cada iteración del algoritmo, está estrategia se conoce como *batch gradient descent*, la cual solo se puede usar en casos en los que el conjunto de entrenamiento se pueda mantener todo el tiempo en memoria.

En otras circunstancias, utilizamos el *stochastic gradient descent* o el *mini-batch gradient descent*. En el primero actualizamos los parámetros de nuestro modelo con cada muestra que procesamos, y en el segundo tomamos subconjuntos de nuestro conjunto para realizar actualizaciones por lotes.
### Stochastic Gradient Descent

In [ ]:
theta_path_sgd = []
m = len(X_b)

n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                  
            y_predict = X_new_b.dot(theta)           
            style = "b-" if i > 0 else "r--"         
            plt.plot(X_new, y_predict, style)        
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 

plt.plot(X, y, "b.")                                 
plt.xlabel("$x_1$", fontsize=18)                     
plt.ylabel("$y$", rotation=0, fontsize=18)           
plt.axis([0, 2, 0, 15])                              
plt.show()                                           

In [ ]:
print(theta)

In [ ]:
predictions = X_b.dot(theta) # Realizar predicciones
mean_squared_error(predictions, y)

### Mini-batch Gradient Descent

In [ ]:
theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

print(theta)

In [ ]:
predictions = X_b.dot(theta) # Realizar predicciones
mean_squared_error(predictions, y)

### Comparando las 3 estrategias

In [ ]:
theta_path_bgd = np.array(theta_path_bgd)
theta_path_sgd = np.array(theta_path_sgd)
theta_path_mgd = np.array(theta_path_mgd)

plt.figure(figsize=(7,4))
plt.plot(theta_path_sgd[:, 0], theta_path_sgd[:, 1], "r-s", linewidth=1, label="Stochastic")
plt.plot(theta_path_mgd[:, 0], theta_path_mgd[:, 1], "g-+", linewidth=2, label="Mini-batch")
plt.plot(theta_path_bgd[:, 0], theta_path_bgd[:, 1], "b-o", linewidth=3, label="Batch")
plt.legend(loc="upper left", fontsize=16)
plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20, rotation=0)
plt.axis([2.5, 4.5, 2.3, 3.9])
plt.show()

### Curvas de entrenamiento

Las curvas de entrenamiento nos permiten evaluar el comportamiento del entrenamiento del modelo a medida que pasa cada iteración o cada época, o a medida que va cambiando algún parámetro.

In [ ]:
np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state=10)

In [ ]:
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train) + 1):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))

    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.legend(loc="upper right", fontsize=14)   
    plt.xlabel("Training set size", fontsize=14) 
    plt.ylabel("RMSE", fontsize=14)   

lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)
plt.axis([0, 80, 0, 3])
plt.show()             

### Acerca del sobreentrenamiento (*Overfitting*)

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler())
    ])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)


sgd_reg = SGDRegressor(max_iter=1, tol=1e-10, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

n_epochs = 500
train_errors, val_errors = [], []
for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))

best_epoch = np.argmin(val_errors)
best_val_rmse = np.sqrt(val_errors[best_epoch])

plt.annotate('Best model',
             xy=(best_epoch, best_val_rmse),
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,
            )

best_val_rmse -= 0.03  # just to make the graph look better
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="Validation set")
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="Training set")
plt.legend(loc="upper right", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("RMSE", fontsize=14)
plt.show()

### Acerca del preprocesamiento y el descenso del gradiente

Para hacer el algoritmo de descenso del gradiente más eficiente, es una práctica común normalizar las escalas en las que se encuentran las características del conjunto de datos, de esta forma se puede lograr un entrenamiento más rápido.

<img src="https://github.com/MoraRubio/ai-learning-resources/blob/main/supervised-learning-ml/src/gradient-descent-feature-scaling.png?raw=true" width="230" align="middle"/>

En nuestro caso, la implementación la podemos realizar con la función `StandardScaler()` de la librería Scikit-learn.